# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("consult_proj").getOrCreate()

In [8]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [3]:
data = spark.read.csv("cruise_ship_info.csv", inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [7]:
data.select("Cruise_line").show()

+-----------+
|Cruise_line|
+-----------+
|    Azamara|
|    Azamara|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
+-----------+
only showing top 20 rows



In [9]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_index").fit(data)

In [10]:
new_data = indexer.transform(data)

In [11]:
new_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [12]:
assembler = VectorAssembler(inputCols=["Age", "Tonnage", "passengers", "length", 
                                       "cabins", "passenger_density", "Cruise_line_index"],
                           outputCol="features")

In [13]:
final_data = assembler.transform(new_data)

In [15]:
final_data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|Cruise_line_index|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|5.063291139240507|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|

In [17]:
final_data = final_data.select("features", "crew")

In [18]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               110|
|   mean| 7.701636363636375|
| stddev|3.4151826491574235|
|    min|               0.6|
|    max|              21.0|
+-------+------------------+



In [20]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                48|
|   mean| 8.006249999999996|
| stddev|3.7264560912480915|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [21]:
lr = LinearRegression(labelCol="crew")

In [22]:
lrModel = lr.fit(train_data)

In [23]:
test_results = lrModel.evaluate(test_data)

In [25]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.2569870475284493|
| 0.09750390473903714|
| -0.7546822520221665|
|-0.28182420616637316|
|  0.4639493806160466|
|-0.31812929609804996|
| -0.5216787906175444|
| -0.4071939307154935|
| -0.6591972407369049|
| -0.5964906928951148|
|-0.23528554597992635|
| 0.21956814347171694|
|-0.48624018056429463|
| 0.20108967784314213|
|-0.21464663118150895|
|-0.44503022421538674|
|  0.7420827241040548|
|  0.9403199183096742|
|   7.102525284377217|
| -1.1324687345936866|
+--------------------+
only showing top 20 rows



In [26]:
test_results.rootMeanSquaredError

1.2700209150689103

In [27]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                48|
|   mean| 8.006249999999996|
| stddev|3.7264560912480915|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [28]:
test_results.r2

0.8813758512291925

In [32]:
print("Varibles : {} \n Coefficients : {}".format(assembler.getInputCols(), lrModel.coefficients))

Varibles : ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'Cruise_line_index'] 
 Coefficients : [-0.00857650625175443,0.017297568993874513,-0.14763128244238435,0.4092567102001556,0.7825169458550433,-0.008558803150310082,0.044056778820557085]


In [33]:
from pyspark.sql.functions import corr

In [34]:
data.select(corr("crew","passengers")).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [35]:
data.select(corr("crew", "cabins")).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

